In [1]:
# Configuration File for Managing API Key as an Environment Variable
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
# Set up LangSmith tracking: https://smith.langchain.com
from langsmith import utils

utils.tracing_is_enabled()

True

In [3]:
#PromptTemplate
from langchain_core.prompts import PromptTemplate

# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method.
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [4]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [5]:
# load openai model

from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Chain Creation

### LCEL (Langchain Expression Language)

chain = prompt | model | output_parser

In [6]:
# Create the prompt as a `PromptTemplate` object.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")


# Combine the prompt and model into a chain
chain = prompt | model

Input values are provided in the form of a Python dictionary (key-value pairs).

When calling the invoke() function, these input values are passed as arguments.

In [8]:
input  = {"topic":"Usability of ai in meidcal term"}
chain.invoke(input)

AIMessage(content='Usability of AI in medical terms refers to how easy and effective it is for healthcare professionals to use artificial intelligence tools in their work. Here are some key points to understand it simply:\n\n1. **User-Friendly Design**: AI tools should be designed so that doctors, nurses, and other healthcare workers can easily understand and use them without extensive training.\n\n2. **Improving Patient Care**: AI can help in diagnosing diseases, predicting patient outcomes, and personalizing treatment plans, making it easier for healthcare providers to give better care.\n\n3. **Efficiency**: AI can automate routine tasks, like data entry or scheduling, allowing healthcare professionals to spend more time with patients and less on administrative work.\n\n4. **Decision Support**: AI can analyze large amounts of medical data quickly, helping doctors make informed decisions based on the latest research and patient information.\n\n5. **Accessibility**: AI tools can be use

In [9]:
answer = chain.stream(input)

In [11]:
#streaming output
# Streaming Output
for token in answer:
    print(token.content, end="", flush=True)

Usability of AI in medical terms refers to how easy and effective it is for healthcare professionals to use artificial intelligence tools in their work. Here are some key points to understand it simply:

1. **User-Friendly Design**: AI tools should be designed so that doctors, nurses, and other healthcare workers can easily understand and operate them without extensive training.

2. **Improving Patient Care**: AI can help in diagnosing diseases, predicting patient outcomes, and personalizing treatment plans, making it easier for healthcare providers to give better care.

3. **Efficiency**: AI can automate routine tasks, like data entry or scheduling, allowing healthcare professionals to spend more time with patients and less on administrative work.

4. **Decision Support**: AI can analyze large amounts of medical data quickly, helping doctors make informed decisions based on the latest research and patient information.

5. **Accessibility**: AI tools can be used in various settings, fr

## Output Parser

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = (
    StrOutputParser()
)  # Directly returns the model's response as a string without modification.

In [13]:
chain  = prompt | model  | output_parser

In [14]:
chain.invoke(input)

'Usability of AI in medical terms refers to how easy and effective it is for healthcare professionals to use artificial intelligence tools and systems in their work. Here are some key points to understand it simply:\n\n1. **User-Friendly Design**: AI tools should be designed so that doctors, nurses, and other healthcare workers can easily understand and operate them without extensive training.\n\n2. **Improving Patient Care**: AI can help in diagnosing diseases, predicting patient outcomes, and personalizing treatment plans, making it easier for healthcare providers to give better care.\n\n3. **Efficiency**: AI can automate routine tasks, like data entry or scheduling, allowing healthcare professionals to focus more on patient care rather than administrative work.\n\n4. **Decision Support**: AI can analyze large amounts of medical data quickly, helping doctors make informed decisions based on the latest research and patient information.\n\n5. **Accessibility**: AI tools can be used in 

## LCEL Interface

---

To make it as easy as possible to create custom chains, we've implemented the `Runnable` protocol.

The `Runnable` protocol is implemented in most components.

It is a standard interface that makes it easy to define custom chains and call them in a standard way. The standard interface includes

- `stream`: Streams a chunk of the response.
- `invoke`: Invoke a chain on an input.
- `batch`: Invoke a chain against a list of inputs.

There are also asynchronous methods

- `astream`: Stream chunks of the response asynchronously.
- `ainvoke`: Invoke a chain asynchronously on an input.
- `abatch`: Asynchronously invoke a chain against a list of inputs.
- `astream_log`: Streams the final response as well as intermediate steps as they occur.



In [ ]:
# Logging your trace using langsmit
import openai
from langsmith import wrappers, traceable

# Auto-trace LLM calls in-context
client = wrappers.wrap_openai(openai.Client())

@traceable # Auto-trace this function
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("Hello, world!")
# Out:  Hello there! How can I assist you today?

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Instantiate the ChatOpenAI model.
model = ChatOpenAI()
# Create a prompt template that asks for jokes on a given topic.
prompt = PromptTemplate.from_template("Describe the {topic} in 3 sentences.")
# Connect the prompt and model to create a conversation chain.
chain = prompt | model | StrOutputParser()

In [ ]:
# Call a function to batch process a given list of topics
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

In [ ]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "multimodal"},
        {"topic": "programming"},
        {"topic": "machineLearning"},
    ],
    config={"max_concurrency": 3},
)

In [16]:
# Use an asynchronous stream to process messages in the 'YouTube' topic.
async for token in chain.astream({"topic": "YouTube"}):
    # Print the message content. Outputs directly without newlines and empties the buffer.
    print(token, end="", flush=True)

YouTube is a popular video-sharing platform where users can upload, view, and comment on a wide variety of content. It has become a major source of entertainment, education, and news for millions of people around the world. With millions of videos available on virtually any topic, YouTube has revolutionized the way we consume media.

## Parallel

Let's take a look at how the LangChain Expression Language supports parallel requests. For example, when you use `RunnableParallel` (often written in dictionary form), you execute each element in parallel.

Here's an example of running two tasks in parallel using the `RunnableParallel` class in the `langchain_core.runnables` module.

Create two chains (`chain1`, `chain2`) that use the `ChatPromptTemplate.from_template` method to get the capital and area for a given `country`.

These chains are connected via the `model` and pipe (`|`) operators, respectively. Finally, we use the `RunnableParallel` class to combine these two chains with the keys `capital` and `area` to create a `combined` object that can be run in parallel.

In [17]:
from langchain_core.runnables import RunnableParallel

# Create a chain that asks for the capital of {country}.
chain1 = (
    PromptTemplate.from_template("What is the capital of {country}?")
    | model
    | StrOutputParser()
)

# Create a chain that asks for the area of {country}.
chain2 = (
    PromptTemplate.from_template("What is the area of {country}?")
    | model
    | StrOutputParser()
)

# Create a parallel execution chain that generates the above two chains in parallel.
combined = RunnableParallel(capital=chain1, area=chain2)

In [18]:
combined

{
  capital: PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')
           | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7e96b1111600>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7e96b1110d90>, root_client=<openai.OpenAI object at 0x7e96b2b2d120>, root_async_client=<openai.AsyncOpenAI object at 0x7e96b1113d00>, model_kwargs={}, openai_api_key=SecretStr('**********'))
           | StrOutputParser(),
  area: PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the area of {country}?')
        | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7e96b1111600>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7e96b1110d90>, root_client=<openai.OpenAI object at 0x7e96b2b2d120>, root_async_client=<openai.AsyncOpenAI object at 0x7e96b1113d00>, model_kw

## Runnable

LangChain's `Runnable` objects provide a modular and flexible approach to designing workflows by enabling the chaining, parallel execution, and transformation of data. These utilities allow for efficient handling of structured inputs and outputs, with minimal code overhead.

Key Components is:

- **`RunnableLambda`**: A lightweight utility that enables the application of custom logic through lambda functions, ideal for dynamic and quick data transformations.
- **`RunnablePassthrough`**: Designed to pass input data unchanged or augment it with additional attributes when paired with the `.assign()` method.
- **`itemgetter`**: A Python `operator` module utility for efficiently extracting specific keys or indices from structured data such as dictionaries or tuples.

These tools can be combined to build powerful workflows, such as:

- Extracting and processing specific data elements using `itemgetter`.
- Performing custom transformations with `RunnableLambda`.
- Creating end-to-end data pipelines with `Runnable` chains.

By leveraging these components, users can design scalable and reusable pipelines for machine learning and data processing workflows.


### RunablePassthrough

`RunnablePassthrough` is a utility designed to streamline data processing workflows by either passing input data unchanged or enhancing it with additional attributes. Its flexibility makes it a valuable tool for handling data in pipelines where minimal transformation or selective augmentation is required.

1. **Simple Data Forwarding**

- Suitable for scenarios where no transformation is required, such as logging raw data or passing it to downstream systems.

2. **Dynamic Data Augmentation**

- Enables the addition of metadata or context to input data for use in machine learning pipelines or analytics systems.
a
---
- `RunnablePassthrough` can either pass the input unchanged or append additional keys to it.
- When `RunnablePassthrough()` is called on its own, it simply takes the input and passes it as is.
- When called using `RunnablePassthrough.assign(...)`, it takes the input and adds additional arguments provided to the assign function.

In [19]:

# Create the prompt and llm
prompt = PromptTemplate.from_template("What is 10 times {num}?")
llm = ChatOpenAI(temperature=0)

# Create the chaina
chain = prompt | llm

In [21]:
from langchain_core.runnables import RunnablePassthrough

#create chain using RunnablePassthrough

runnable_chain = {"num":RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [22]:
runnable_chain

{
  num: RunnablePassthrough()
}
| PromptTemplate(input_variables=['num'], input_types={}, partial_variables={}, template='What is 10 times {num}?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7e96b118ae30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7e96b118bf10>, root_client=<openai.OpenAI object at 0x7e96b11894e0>, root_async_client=<openai.AsyncOpenAI object at 0x7e96b118ae00>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [23]:
runnable_chain.invoke(2)

'10 times 2 is equal to 20.'

In [29]:
# assign new key pair using .assign()

(RunnablePassthrough.assign(num=lambda x: x["num"] * 3)).invoke({"num": 1})


{'num': 3}

In [28]:
runnable_chain_assign.invoke({"num": 1})

"{'num': 2, 'num': 2, 'num': 2, 'num': 2, 'num': 2, 'num': 2, 'num': 2, 'num': 2, 'num': 2, 'num': 2}"

In [31]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# Function that returns the length of a sentence.
def length_function(text):
    return len(text)


# Function that returns the product of the lengths of two sentences.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# Function that uses _multiple_length_function to return the product of the lengths of two sentences.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("What is {a} + {b}?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [32]:
chain

{
  a: RunnableLambda(itemgetter('word1'))
     | RunnableLambda(length_function),
  b: {
       text1: RunnableLambda(itemgetter('word1')),
       text2: RunnableLambda(itemgetter('word2'))
     }
     | RunnableLambda(multiple_length_function)
}
| ChatPromptTemplate(input_variables=['a', 'b'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['a', 'b'], input_types={}, partial_variables={}, template='What is {a} + {b}?'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7e96a49a5600>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7e96a49a7430>, root_client=<openai.OpenAI object at 0x7e96b0fb77f0>, root_async_client=<openai.AsyncOpenAI object at 0x7e96a49a55d0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

## Prompt Template

In [33]:
# Define template
template = "What are the capitals of {country1} and {country2}, respectively?"

# Create a prompt template with `PromptTemplate` object
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "United States of America"  # Pass `partial_variables` in dictionary form
    },
)
prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': 'United States of America'}, template='What are the capitals of {country1} and {country2}, respectively?')

In [34]:
prompt.format(country1="Nepal")


'What are the capitals of Nepal and United States of America, respectively?'

In [35]:
prompt_partial = prompt.partial(country2="India")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': 'India'}, template='What are the capitals of {country1} and {country2}, respectively?')

In [36]:
chain = prompt_partial | llm


In [37]:
chain.invoke("nepal").content

'The capital of Nepal is Kathmandu, and the capital of India is New Delhi.'

#### Using partial_variables

Using `partial_variables` , you can partially apply functions.  This is particularly useful when there are **common variables** to be shared.

Common examples are **date or time**.

Suppose you want to specify the current date in your prompt, hardcoding the date into the prompt or passing it along with other input variables may not be practical. In this case, using a function that returns the current date to modify the prompt partially is much more convenient.

In [39]:
from datetime import datetime

# Print the current date
datetime.now().strftime("%B %d")


# Define function to return the current date
def get_today():
    return datetime.now().strftime("%B %d")


prompt = PromptTemplate(
    template="Today's date is {today}. Please list {n} celebrities whose birthday is today. Please specify their date of birth.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # Pass `partial_variables` in dictionary form
    },
)

In [40]:
prompt.format(n=1)

"Today's date is January 20. Please list 1 celebrities whose birthday is today. Please specify their date of birth."

## Load prompt template from YAML file

You can manage prompt templates in seperate yaml files and load using `load_prompt` .

In [43]:

from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

No `_type` key found, defaulting to `prompt`.


ValidationError: 2 validation errors for PromptTemplate
input_variables
  Field required [type=missing, input_value={'prompt_template': {'tem...ables': ['user_input']}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
template
  Field required [type=missing, input_value={'prompt_template': {'tem...ables': ['user_input']}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

### ChatPromptTemplate

`ChatPromptTemplate` can be used to include a conversation history as a prompt.

Messages are structured as tuples in the format (`role` , `message` ) and are created as a list.

**role**
- `"system"` : A system setup message, typically used for global settings-related prompts.
- `"human"` : A user input message.
- `"ai"` : An AI response message.

In [44]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("What is the capital of {country}?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?'), additional_kwargs={})])

In [45]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "You are a friendly AI assistant. Your name is {name}."),
        ("human", "Nice to meet you!"),
        ("ai", "Hello! How can I assist you?"),
        ("human", "{user_input}"),
    ]
)

# Create chat messages
messages = chat_template.format_messages(name="Teddy", user_input="What is your name?")
messages

[SystemMessage(content='You are a friendly AI assistant. Your name is Teddy.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Nice to meet you!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

In [47]:
chain = chat_template | llm
chain.invoke({"name":"Anil","user_input":"What is your name?"})

AIMessage(content='My name is Anil. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 50, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a823204c-0717-441f-aba6-2b7e033521c8-0', usage_metadata={'input_tokens': 50, 'output_tokens': 14, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Few Shot Templates

LangChain's Few-Shot Prompting provides a robust framework for guiding language models to generate high-quality outputs by supplying carefully selected examples. This technique minimizes the need for extensive model fine-tuning while ensuring precise, context-aware results across diverse use cases.


Few-Shot Prompt Templates: Define the structure and format of prompts by embedding illustrative examples, guiding the model to produce consistent outputs.

Example Selection Strategies: Dynamically select the most relevant examples for a given query, enhancing the model's contextual understanding and response accuracy.

Chroma Vector Store: A powerful utility for storing and retrieving examples based on semantic similarity, enabling scalable and efficient prompt construction.

### FewShotPromptTemplate

In [49]:
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(
    temperature=0,  # Creativity
    model_name="gpt-4o-mini",  # Use a valid model name
)

# User query
question = "What is the capital of United States of America?"

# Query the model
response = llm.invoke(question)

# Print the response
print(response)

content='The capital of the United States of America is Washington, D.C.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 17, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None} id='run-b4691eba-3a93-43d9-9be3-4b31843f889e-0' usage_metadata={'input_tokens': 17, 'output_tokens': 15, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [50]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# Define examples for the few-shot prompt
examples = [
    {
        "question": "Who lived longer, Steve Jobs or Einstein?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein
""",
    },
    {
        "question": "When was the founder of Naver born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967
""",
    },
    {
        "question": "Who was the reigning king when Yulgok Yi's mother was born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang born?
Intermediate Answer: Shin Saimdang was born in 1504.
Additional Question: Who was the king of Joseon in 1504?
Intermediate Answer: The king of Joseon in 1504 was Yeonsangun.
The final answer is: Yeonsangun
""",
    },
    {
        "question": "Are the directors of Oldboy and Parasite from the same country?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the director of Oldboy?
Intermediate Answer: The director of Oldboy is Park Chan-wook.
Additional Question: Which country is Park Chan-wook from?
Intermediate Answer: Park Chan-wook is from South Korea.
Additional Question: Who is the director of Parasite?
Intermediate Answer: The director of Parasite is Bong Joon-ho.
Additional Question: Which country is Bong Joon-ho from?
Intermediate Answer: Bong Joon-ho is from South Korea.
The final answer is: Yes
""",
    },
]

In [51]:
# Create an example prompt template
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

# Print the first formatted example
print(example_prompt.format(**examples[0]))

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein



In [52]:
# Initialize the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# Example question
question = "How old was Bill Gates when Google was founded?"

# Generate the final prompt
final_prompt = few_shot_prompt.format(question=question)
print(final_prompt)

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein


Question:
When was the founder of Naver born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967


Question:
Who was the reigning king when Yulgok Yi's mother was born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang bo

In [58]:
chain =  few_shot_prompt | llm | StrOutputParser()
chain.invoke({"question": "How old was Bill Gates when Google was founded?"} )

'Does this question require additional questions: Yes.  \nAdditional Question: When was Google founded?  \nIntermediate Answer: Google was founded on September 4, 1998.  \nAdditional Question: What year was Bill Gates born?  \nIntermediate Answer: Bill Gates was born on October 28, 1955.  \nAdditional Question: How old was Bill Gates on September 4, 1998?  \nIntermediate Answer: Bill Gates was 42 years old when Google was founded.  \nThe final answer is: 42 years old.'

## Dynamic Exmple Selection with Chroma

In [59]:
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import ChatOpenAI

# Initialize embeddings and vector store
embeddings = OpenAIEmbeddings()
chroma = Chroma(persist_directory="example_selector", embedding_function=embeddings)

In [60]:
examples = [
    {
        "question": "Who lived longer, Steve Jobs or Einstein?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein
""",
    },
    {
        "question": "When was the founder of Google born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the founder of Google?
Intermediate Answer: Google was founded by Larry Page and Sergey Brin.
Additional Question: When was Larry Page born?
Intermediate Answer: Larry Page was born on March 26, 1973.
Additional Question: When was Sergey Brin born?
Intermediate Answer: Sergey Brin was born on August 21, 1973.
The final answer is: Larry Page was born on March 26, 1973, and Sergey Brin was born on August 21, 1973.
""",
    },
    {
        "question": "Who was the President when Donald Trump's mother was born?",  # 쉼표 추가
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is Donald Trump's mother?
Intermediate Answer: Donald Trump's mother is Mary Anne MacLeod Trump.
Additional Question: When was Mary Anne MacLeod Trump born?
Intermediate Answer: Mary Anne MacLeod Trump was born on May 10, 1912.
Additional Question: Who was the U.S. President in 1912?
Intermediate Answer: William Howard Taft was President in 1912.
The final answer is: William Howard Taft
""",
    },
    {
        "question": "Are the directors of Oldboy and Parasite from the same country?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the director of Oldboy?
Intermediate Answer: The director of Oldboy is Park Chan-wook.
Additional Question: Which country is Park Chan-wook from?
Intermediate Answer: Park Chan-wook is from South Korea.
Additional Question: Who is the director of Parasite?
Intermediate Answer: The director of Parasite is Bong Joon-ho.
Additional Question: Which country is Bong Joon-ho from?
Intermediate Answer: Bong Joon-ho is from South Korea.
The final answer is: Yes
""",
    },
]

In [62]:
texts = [example["question"] for example in examples]
metadatas = [example for example in examples]
metadatas

[{'question': 'Who lived longer, Steve Jobs or Einstein?',
  'answer': 'Does this question require additional questions: Yes.\nAdditional Question: At what age did Steve Jobs die?\nIntermediate Answer: Steve Jobs died at the age of 56.\nAdditional Question: At what age did Einstein die?\nIntermediate Answer: Einstein died at the age of 76.\nThe final answer is: Einstein\n'},
 {'question': 'When was the founder of Google born?',
  'answer': 'Does this question require additional questions: Yes.\nAdditional Question: Who is the founder of Google?\nIntermediate Answer: Google was founded by Larry Page and Sergey Brin.\nAdditional Question: When was Larry Page born?\nIntermediate Answer: Larry Page was born on March 26, 1973.\nAdditional Question: When was Sergey Brin born?\nIntermediate Answer: Sergey Brin was born on August 21, 1973.\nThe final answer is: Larry Page was born on March 26, 1973, and Sergey Brin was born on August 21, 1973.\n'},
 {'question': "Who was the President when Don

In [63]:
# Add examples to the vector store
texts = [example["question"] for example in examples]
metadatas = [example for example in examples]
chroma.add_texts(texts=texts, metadatas=metadatas)

# Set up Example Selector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=chroma,  # Only vectorstore is needed
    k=1  # Number of examples to select
)

# Define Few-Shot Prompt Template
example_prompt_template = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}\n"
)
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_template,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)


In [68]:
query = {"question": "Is Nepal larger than America?"}
formatted_prompt = prompt.format(**query)

In [69]:
formatted_prompt

'Question:\nWho lived longer, Steve Jobs or Einstein?\nAnswer:\nDoes this question require additional questions: Yes.\nAdditional Question: At what age did Steve Jobs die?\nIntermediate Answer: Steve Jobs died at the age of 56.\nAdditional Question: At what age did Einstein die?\nIntermediate Answer: Einstein died at the age of 76.\nThe final answer is: Einstein\n\n\n\nQuestion:\nIs Nepal larger than America?\nAnswer:'

In [71]:
response = llm.invoke(formatted_prompt)

In [73]:
response.content

'Does this question require additional questions: Yes.  \nAdditional Question: What are the total land areas of Nepal and America?  \nIntermediate Answer: Nepal has a land area of approximately 147,516 square kilometers, while the United States has a land area of about 9.8 million square kilometers.  \nThe final answer is: No, Nepal is not larger than America.'

In [ ]:
from langchain import hub
from langchain.prompts import PromptTemplate

prompt_title = "llm-response-evaluation"

evaluation_prompt = """Evaluate the LLM's response based on the following criteria:

INPUT:
Question: {question}
Context: {context}
LLM Response: {answer}

EVALUATION CRITERIA:
1. Accuracy (0-10)
- Perfect (10): Completely accurate, perfectly aligned with context
- Good (7-9): Minor inaccuracies
- Fair (4-6): Some significant inaccuracies
- Poor (0-3): Major inaccuracies or misalignment

2. Completeness (0-10)
- Perfect (10): Comprehensive coverage of all relevant points
- Good (7-9): Covers most important points
- Fair (4-6): Missing several key points
- Poor (0-3): Critically incomplete

3. Context Relevance (0-10)
- Perfect (10): Optimal use of context
- Good (7-9): Good use with minor omissions
- Fair (4-6): Partial use of relevant context
- Poor (0-3): Poor context utilization

4. Clarity (0-10)
- Perfect (10): Exceptionally clear and well-structured
- Good (7-9): Clear with minor issues
- Fair (4-6): Somewhat unclear
- Poor (0-3): Confusing or poorly structured

SCORING METHOD:
1. Calculate individual scores
2. Compute weighted average:
   - Accuracy: 40%
   - Completeness: 25%
   - Context Relevance: 25%
   - Clarity: 10%
3. Normalize to 0-1 scale

OUTPUT FORMAT:
{
    "individual_scores": {
        "accuracy": float,
        "completeness": float,
        "context_relevance": float,
        "clarity": float
    },
    "weighted_score": float,
    "normalized_score": float,
    "evaluation_notes": string
}

Return ONLY the normalized_score as a decimal between 0 and 1."""

prompt = PromptTemplate.from_template(evaluation_prompt)

In [ ]:
PROMPT_TEMPLATE = """
As a medical professional, analyze clinical cases with:

INPUT:
- Patient Information: {patient_data}
- Clinical Notes: {clinical_notes}

PROVIDE:
1. Clinical Assessment
2. Diagnostic Process
3. Treatment Plan
4. Risk Assessment

OUTPUT FORMAT:
{
    "clinical_summary": str,
    "differential_diagnosis": list,
    "treatment_plan": dict,
    "risk_assessment": dict
}
"""